In [1]:
# checkpoint表示要使用哪个预训练模型
import torch
checkpoint = "hfl/chinese-roberta-wwm-ext"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

Using cuda device


In [2]:
from torch.utils.data import Dataset
categories = set()
class ReadData(Dataset):
    def __init__(self, data_file):
        self.data = self.load_data(data_file)
    
    def load_data(self, data_file):
        Data = {}
        with open(data_file, 'rt', encoding='utf-8') as f:
            for idx, line in enumerate(f.read().split('\n\n')):
                if not line:
                    break
                sentence, tags = '', []
                for i, c in enumerate(line.split('\n')):
                    word, tag = c.split('\t')
                    if word==' ':
                        word='1'
                    sentence += word
                    if tag[0] == 'B':
                        tags.append([i, i, word, tag[2:]]) # Remove the B- or I-
                        # print(tags)
                        categories.add(tag[2:])
                    elif tag[0] == 'I':
                        # print(tags)
                        # print(word)
                        # print(tag)
                        tags[-1][1] = i
                        tags[-1][2] += word
                Data[idx] = {
                    'sentence': sentence, 
                    'tags': tags
                }
        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
# from transformers import BertTokenizerFast
import numpy as np

# tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def collote_fn(batch_samples):
    # batch_sentence, batch_tags,mask = [], [], []
    batch_sentence, batch_tags = [], []
    for sample in batch_samples:
        # print(sample)
        batch_sentence.append(sample['sentence'])
        batch_tags.append(sample['tags'])
        # mask.append(sample['mask_tensor'])
    batch_inputs = tokenizer(
        batch_sentence, 
        padding=True, 
        truncation=True, 
        return_tensors="pt",
        # max_length=256
    )
    batch_label = np.zeros(batch_inputs['input_ids'].shape, dtype=int)
    for s_idx, sentence in enumerate(batch_sentence):
        encoding = tokenizer(sentence, truncation=True)
        batch_label[s_idx][0] = 0
        batch_label[s_idx][len(encoding.tokens())-1:] = 0
        for char_start, char_end, _, tag in batch_tags[s_idx]:
            token_start = encoding.char_to_token(char_start)
            token_end = encoding.char_to_token(char_end)
            batch_label[s_idx][token_start] = label2id[f"B-{tag}"]
            batch_label[s_idx][token_start+1:token_end+1] = label2id[f"I-{tag}"]
    return batch_inputs, torch.tensor(batch_label)

In [4]:
from torch import nn
from transformers import AutoModel, BertConfig
from torchcrf import CRF

class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.config=BertConfig.from_pretrained(checkpoint)
        self.bert = AutoModel.from_pretrained(checkpoint, config = self.config)
        self.BiLstm=nn.LSTM(input_size=self.config.hidden_size,hidden_size=512,batch_first=True,bidirectional=True,num_layers=2)
        self.Linear = nn.Linear(512*2, len(id2label))
        self.crf = CRF(len(id2label),batch_first=True)
            
    # def forward(self, x):
    def forward(self, x, y):
        # 1.
        # output = self.bert(**x).last_hidden_state
        # output, _ = self.BiLstm(output)
        # output = self.Linear(output)
        # return output
        # 2.
        output = self.bert(**x).last_hidden_state
        output, _ = self.BiLstm(output)
        output = self.Linear(output)
        # loss = self.crf(emissions=output,tags=y,mask=mask_tensor)
        # tag = self.crf.decode(emissions=output,,mask=mask_tensor)
        loss = self.crf(emissions=output,tags=y)
        tag = self.crf.decode(emissions=output)
        tag=torch.tensor(tag)
        return loss, tag
    
    def decode(self,x):
        output = self.bert(**x).last_hidden_state
        output, _ = self.BiLstm(output)
        output = self.Linear(output)
        tag = self.crf.decode(emissions=output)
        tag=torch.tensor(tag)
        return tag

In [5]:
from tqdm.auto import tqdm
def train_loop(dataloader, model, loss_fn, optimizer, lr_scheduler, epoch, total_loss):
# def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1) * len(dataloader)
    model.train()
    for batch, (X, y) in enumerate(dataloader, start=1):
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)
        loss, tag = model(X, y)
        #  通过 pred.permute(0, 2, 1) 交换后两维，将模型预测结果从(batch,seq,7) 调整为 (batch,7,seq)。
        # loss = loss_fn(pred.permute(0, 2, 1), y)
        loss = abs(loss)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
    return total_loss

In [6]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
def test_loop(dataloader, model):
    true_labels, true_predictions = [], []

    model.eval()
    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)
            
            # pred = model(X)
            loss, tag = model(X, y)
            
            # predictions = pred.argmax(dim=-1)
            predictions = tag
            
            true_labels += [[id2label[int(l)] for l in label if l != -100] for label in y]
            true_predictions += [
                [id2label[int(p)] for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, y)
            ]
    print(classification_report(true_labels, true_predictions, mode='strict', scheme=IOB2))

In [7]:
MyData = ReadData('./example.train')
valid_size=int(len(MyData)*0.05)
train_size=len(MyData)-valid_size
train_data, valid_data = torch.utils.data.random_split(MyData, [train_size, valid_size])
print(train_data[0])
# print(len(train_data))
# print(len(valid_data))

# train_data = ReadData('./example.train')
# valid_data = ReadData('./example.dev')
# test_data = ReadData('./example.test')
# print(train_data[0])

{'sentence': '漏油。套管漏油。更换密封件', 'tags': [[0, 1, '漏油', 'Pro'], [3, 6, '套管漏油', 'Cau'], [8, 12, '更换密封件', 'Met']]}


In [8]:
print('categories\n',categories)

id2label = {0:'O'}
for c in list(sorted(categories)):
    id2label[len(id2label)] = f"B-{c}"
    id2label[len(id2label)] = f"I-{c}"
label2id = {v: k for k, v in id2label.items()}

print('id2label\n',id2label)
print('label2id\n',label2id)

categories
 {'Pro', 'Cau', 'Met'}
id2label
 {0: 'O', 1: 'B-Cau', 2: 'I-Cau', 3: 'B-Met', 4: 'I-Met', 5: 'B-Pro', 6: 'I-Pro'}
label2id
 {'O': 0, 'B-Cau': 1, 'I-Cau': 2, 'B-Met': 3, 'I-Met': 4, 'B-Pro': 5, 'I-Pro': 6}


In [9]:
# train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=collote_fn)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=4, shuffle=False, collate_fn=collote_fn)
# test_dataloader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=collote_fn)

batch_X, batch_y = next(iter(train_dataloader))
# print('batch_X shape:', {k: v.shape for k, v in batch_X.items()})
# print('batch_y shape:', batch_y.shape)
# print(batch_X)
# print(batch_y)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
model = model().to(device)
# print(model)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
from transformers import AdamW, get_scheduler
learning_rate = 1e-5
epoch_num = 300
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=learning_rate)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
loss_list=[]
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    # total_loss = train_loop(train_dataloader, model, optimizer, lr_scheduler, t+1, total_loss)
    if(t%30==0):
        test_loop(valid_dataloader, model)

print("Done!")

Epoch 1/300
-------------------------------


/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/88 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:333.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


  0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 275/300
-------------------------------


  0%|          | 0/88 [00:00<?, ?it/s]

Epoch 276/300
-------------------------------


  0%|          | 0/88 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
sentence = '某风电场地处戈壁滩，环境温差较大，安装某厂家生产的具有独立布置高低压开关室的“紧凑型箱变”100台，\
投运1年以来发生故障20台次，统计见表2。分析认为，该箱变的油箱设计体积不够，不满足当地环境温差的要求。'

results = []
with torch.no_grad():
    inputs = tokenizer(sentence, truncation=True, return_tensors="pt")
    inputs = inputs.to(device)
    pred = model.decode(inputs)
    # print(pred)
    predictions = pred[0].tolist()
    pred_label = []
    inputs_with_offsets = tokenizer(sentence, return_offsets_mapping=True)
    tokens = inputs_with_offsets.tokens()
    offsets = inputs_with_offsets["offset_mapping"]
    idx = 0
    while idx < len(predictions):
        pred = predictions[idx]
        label = id2label[pred]
        if label != "O":
            label = label[2:] # Remove the B- or I-
            start, end = offsets[idx]
            while (
                idx + 1 < len(predictions) and 
                id2label[predictions[idx + 1]] == f"I-{label}"
            ):
                # all_scores.append(probabilities[idx + 1][predictions[idx + 1]])
                _, end = offsets[idx + 1]
                idx += 1
            word = sentence[start:end]
            pred_label.append(
                {
                    "entity_group": label,
                    "word": word,
                    "start": start,
                    "end": end,
                }
            )
        idx += 1
    print(pred_label)

[{'entity_group': 'Pro', 'word': '环境温差较大', 'start': 10, 'end': 16}, {'entity_group': 'Met', 'word': '某厂家生产的具有独立布置高低压开关室的“紧凑型箱变', 'start': 19, 'end': 44}, {'entity_group': 'Cau', 'word': '油箱设计体积不够', 'start': 80, 'end': 88}, {'entity_group': 'Cau', 'word': '不满足当地环境', 'start': 89, 'end': 96}]


In [13]:
test_loop(valid_dataloader, model)

  0%|          | 0/10 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         Cau       0.32      0.44      0.37        18
         Met       0.38      0.38      0.38        29
         Pro       0.74      0.73      0.74        44

   micro avg       0.53      0.56      0.54        91
   macro avg       0.48      0.52      0.50        91
weighted avg       0.54      0.56      0.55        91

